# Minimal with visualization

## imports

In [ ]:
import os
from minimal.solver import Solver
from minimal import armatures
from minimal.models import KinematicModel, KinematicPCAWrapper
import numpy as np
import minimal.config as config
from dataloader.result_loader import KinectResultLoader
from minimal.bridge import JointsBridge
import torch
from pytorch3d.structures import Pointclouds

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['savefig.dpi'] = 80
mpl.rcParams['figure.dpi'] = 80

from visualization.utils import o3d_plot, o3d_coord, o3d_mesh, o3d_pcl, o3d_skeleton
import open3d as o3d

from dataloader.utils import ymdhms_time

%load_ext autoreload
%autoreload 2

np.random.seed(20160923)

## Single Frame Minimal

In [ ]:
n_pose = 23 * 3 # degrees of freedom, (n_joints - 1) * 3
# smpl 1.0.0: 10
# smpl 1.1.0: 300
n_shape = 10

input_path = '/media/nesc525/perple/2021-08-18_10-49-44-T'

In [ ]:
k_loader = KinectResultLoader(input_path)
files = k_loader.select_item(298, "id")
kinect_skeleton = np.load(files["kinect/master/skeleton"]["filepath"])
kinect_pcls = np.vstack(np.load(files['kinect/master/pcls']["filepath"]))

bridge = JointsBridge()
kpts_gt, pcl_gt, origin_scale = bridge.smpl_from_kinect(kinect_skeleton[0], kinect_pcls)

mesh = KinematicModel(config.SMPL_MODEL_1_0_MALE_PATH, armatures.SMPLArmature, scale=1)

wrapper = KinematicPCAWrapper(mesh, n_pose=n_pose)
solver = Solver(wrapper, max_iter=int(10e7))

mesh_init, kpts_init = wrapper.run(np.zeros(wrapper.n_params))

In [ ]:
o3d_plot([o3d_pcl(kpts_gt, [0,0,1]), o3d_pcl(pcl_gt, [1,0,0]), o3d_pcl(kpts_init, [0,1,0])], 'Minimal Input')

In [ ]:
params_est = solver.solve_full(kpts_gt, pcl_gt, origin_scale, verbose=1, mse_threshold=1e-5)

In [ ]:
shape_est, pose_pca_est, pose_glb_est = wrapper.decode(params_est)

print('----------------------------------------------------------------------')
print('estimated parameters')
print('pose pca coefficients:', pose_pca_est)
print('pose global rotation:', pose_glb_est)
print('shape: pca coefficients:', shape_est)

mesh.set_params(pose_pca=pose_pca_est)
mesh.save_obj(os.path.join(config.SAVE_PATH, './esttm={}.obj'.format(ymdhms_time())))

## Stream Minimal